In [1]:
# Import dependencies
import requests
import numpy as np
import pandas as pd
from config import auth

In [2]:
# Set authorization header using encoded spotify authentication
headers = {'Authorization': 'Basic ' + auth}

# Set data for making call to spotify
data = [('grant_type', 'client_credentials')]

# Make post request to get token
response = requests.post('https://accounts.spotify.com/api/token', headers = headers, data = data).json()

token = response['access_token']

# Set authorization header using token
headers = {'Authorization': 'Bearer ' + token}

In [3]:
# Spotify endpoint to get all "categories" // moods
mood_endpoint = "https://api.spotify.com/v1/browse/categories"

# Make get request
mood_response = requests.get(mood_endpoint, headers = headers).json()

# I remember us mentioning possibly looking at moods, so while
# I'm not using this downstream, I figured I would leave it to
# build on later if we have the opportunity.

In [4]:
# Get all playlist names within spotify "Top List" category
toplist_endpoint = "https://api.spotify.com/v1/browse/categories/toplists/playlists"

# Make get request
toplist_response = requests.get(toplist_endpoint, headers = headers).json()

# Make variable to shorten typing downstream
playlists = toplist_response['playlists']['items']

# Collect track lists for each playlist
toplists_tracks = {}

# for each number in array of digits 0 - number of playlists
for i in np.arange(len(playlists)):
    # URL to fetch spotify track ids for songs in each playlist
    tracklist_endpoint = playlists[i]['tracks']['href']

    # Make get request
    tracklist_response = requests.get(tracklist_endpoint, headers = headers).json()
    
    # Collect track ids
    track_id = []
    
    # for each number in array of digits 0 - number of tracks in playlist
    for q in np.arange(len(tracklist_response['items'])):
        # Append track id to list
        track_id.append(tracklist_response['items'][q]['track']['id'])
    
    # Add dictionary entry of playlist with value of list of track ids
    toplists_tracks[playlists[i]['name']] = track_id

In [5]:
# Get all song data for songs within toplists_tracks
playlists_data = []
for key, value in toplists_tracks.items():
    
    # Initialize lists
    playlist = key
    track_name = []
    artist_name = []
    duration = []
    popularity = []
    instrumentalness = []
    
    # For each number in array of digits 0 - number of tracks in track list 
    for i in np.arange(len(value)):
        # Paste together track data endpoint for each track
        track_endpoint = f"https://api.spotify.com/v1/tracks/{value[i]}"
        
        # Make get request
        track_response = requests.get(track_endpoint, headers = headers).json()
        
        # Extract data to lists
        track_name.append(track_response['name'])
        artist_name.append(track_response['artists'][0]['name'])  # Only considering primary artist
        duration.append(track_response['duration_ms'])
        popularity.append(track_response['popularity'])
        
        # Paste together audio features endpoint for each track
        audio_endpoint = f"https://api.spotify.com/v1/audio-features/{value[i]}"
        
        try:
            # Make get request
            audio_response = requests.get(audio_endpoint, headers = headers).json()
            instrumentalness.append(audio_response['instrumentalness'])
            
        except KeyError:
            print(f"No audio features page for track {track_response['name']} by {track_response['artists'][0]['name']}")
            instrumentalness.append("NaN")
        
    # Gather information into dictionary and append that dictionary to playlists_data
    track_dict = {"Playlist": playlist,
                  "Track Name": track_name,
                  "Artist Name": artist_name,
                  "Duration (ms)": duration,
                  "Popularity": popularity,
                  "Instrumentalness": instrumentalness}
    playlists_data.append(track_dict)

ConnectionError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/tracks/0ATmjiSoHDCHeeQ9FcvZOW (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001F54F887198>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [ ]:
# Dataframes by playlist
today_top_hits = pd.DataFrame(playlists_data[0])
today_top_hits.to_csv("../Data_Files/todays_top_hits.csv", index = False)

rap_caviar = pd.DataFrame(playlists_data[1])
today_top_hits.to_csv("../Data_Files/todays_top_hits.csv", index = False)

mint = pd.DataFrame(playlists_data[2])
rap_caviar.to_csv("../Data_Files/rap_caviar.csv", index = False)

rock_this = pd.DataFrame(playlists_data[3])
rock_this.to_csv("../Data_Files/rock_this.csv", index = False)

are_and_be = pd.DataFrame(playlists_data[4])
are_and_be.to_csv("../Data_Files/are_and_be.csv", index = False)

viva_latino = pd.DataFrame(playlists_data[5])
viva_latino.to_csv("../Data_Files/viva_latino.csv", index = False)

hot_country = pd.DataFrame(playlists_data[6])
hot_country.to_csv("../Data_Files/hot_country.csv", index = False)

new_music_friday_malaysia = pd.DataFrame(playlists_data[7])
new_music_friday_malaysia.to_csv("../Data_Files/new_music_friday_malaysia.csv", index = False)

viral_hits = pd.DataFrame(playlists_data[8])
viral_hits.to_csv("../Data_Files/viral_hits.csv", index = False)

fresh_finds = pd.DataFrame(playlists_data[9])
fresh_finds.to_csv("../Data_Files/fresh_finds.csv", index = False)

global_top_50 = pd.DataFrame(playlists_data[10])
global_top_50.to_csv("../Data_Files/global_top_50.csv", index = False)

united_states_top_50 = pd.DataFrame(playlists_data[11])
united_states_top_50.to_csv("../Data_Files/united_states_top_50.csv", index = False)

global_viral_50 = pd.DataFrame(playlists_data[12])
global_viral_50.to_csv("../Data_Files/global_viral_50.csv", index = False)

united_states_viral_50 = pd.DataFrame(playlists_data[13])
united_states_viral_50.to_csv("../Data_Files/united_states_viral_50.csv", index = False)